In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
tf.reset_default_graph
num_class = 10
unit = 512
lr = 0.0001
x = tf.placeholder(tf.float32, [None, 28 * 28])
y = tf.placeholder(tf.float32, [None, num_class])

weight1 = tf.Variable(tf.random_normal([28*28,unit] , stddev=0.3))
bias1    =tf.Variable(tf.random_normal([unit], stddev=0.3))
hidden1  =tf.add(tf.matmul(x, weight1), bias1)
act1   =tf.nn.relu(hidden1)


weight2 = tf.Variable(tf.random_normal([unit,unit], stddev=0.3))
bias2    =tf.Variable(tf.random_normal([unit], stddev=0.3))
hidden2  =tf.add(tf.matmul(act1, weight2), bias2)
act2   =tf.nn.relu(hidden2)

weight3 = tf.Variable(tf.random_normal([unit,num_class], stddev=0.3))
bias3    =tf.Variable(tf.random_normal([num_class], stddev=0.3))
hidden3  =tf.add(tf.matmul(act2, weight3), bias3)
y_   =tf.nn.softmax(hidden3)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,
                    logits = y_))

'''
l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.5, scope=None)

regularizer_l1= tf.contrib.layers.apply_regularization(l1_regularizer, [weight1, weight2, weight3, weight4])

cost = cost +regularizer_l1 




l2_regularizer = tf.contrib.layers.l2_regularizer(scale=0.3, scope=None)

regularizer_l2= tf.contrib.layers.apply_regularization(l2_regularizer, [weight1])

cost = cost + regularizer_l2 

'''

optimizer = tf.train.AdamOptimizer(learning_rate= lr ).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost)
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  


init=tf.global_variables_initializer()
#session_conf=tf.ConfigProto(intra_op_parallelism_threads=4)
#sess=tf.Session(config=session_conf)
sess=tf.Session()
sess.run(init)
epoach= 100
batch = 150
val_acc_list = []
val_loss_list = []
test_acc_list = []
test_loss_list = []
for i in range(epoach):
    train_acc=0
    for _ in range(int(mnist.train.num_examples/batch)): 
        batch_x , batch_y =mnist.train.next_batch(batch)
        [c,loss,acc]=sess.run([optimizer,cost,accuracy],feed_dict={x: batch_x , y: batch_y})
    loss,acc=sess.run([cost,accuracy ] ,feed_dict={x: mnist.validation.images, y: mnist.validation.labels})
    print(i,'validation accuracy :',acc)
    val_acc_list.append(acc)
    val_loss_list.append(loss)
    loss, acc=sess.run([cost,accuracy ], feed_dict = {x: mnist.test.images, y: mnist.test.labels} )    
    print(i,'test accuracy :',acc)
    test_acc_list.append(acc)
    test_loss_list.append(loss)

w1 = tf.Print(weight1,[weight1])
w1 = np.array(w1.eval(session=sess))
w2 = tf.Print(weight2,[weight2])
w2 = np.array(w2.eval(session=sess))

w3 = tf.Print(weight3,[weight3])
w3 = np.array(w3.eval(session=sess))
sess.close() 



In [0]:
      

plt.figure(2)
w1 = np.ravel(w1)
plt.title("histogram of h1")
plt.xlabel("value")
plt.ylabel("number")
plt.hist(w1, bins=100, range=(-3, 3))

plt.figure(3)
w2 = np.ravel(w2)
plt.title("histogram of h2")
plt.xlabel("value")
plt.ylabel("number")
plt.hist(w2, bins=100, range=(-3, 3))

plt.figure(4)
w3 = np.ravel(w3)
plt.title("histogram of h3")
plt.xlabel("value")
plt.ylabel("number")
plt.hist(w3, bins=100, range=(-3, 3))

plt.show()  
    
    

In [0]:

plt.plot(range(epoach) , val_acc_list , label ='val')
plt.plot(range(epoach) , test_acc_list , label='test')
plt.show()
plt.plot(range(epoach) ,val_loss_list  ,label='val')
plt.plot(range(epoach) , test_loss_list , label='test')
plt.show()



In [0]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

In [0]:
!tar -zxvf cifar-10-python.tar.gz

In [0]:
import numpy as np  
import os  
import matplotlib.pyplot as plt 
%matplotlib inline
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow.contrib.slim as slim


In [0]:

def one_hot(x,n):
    x=np.array(x)
    assert x.ndim ==1
    return np.eye(n)[x]

def load_batch_cifar10(filename):
    path =os.path.join('./cifar-10-batches-py/' , filename)
    batch =np.load(path)
    data=batch['data']/255.0
    labels=one_hot(batch['labels'],n=10)
    return data.astype('float32') , labels.astype('float32')

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def cifar10():
    train_x=[]
    train_y=[]
    print("loading train...")
    for k in range(1,6):
        dict_ =unpickle('./cifar-10-batches-py/data_batch_%d' % k )
        train_x.append(dict_[b'data'])
        train_y.append(one_hot (dict_[b'labels'] ,10))
    train_x = np.concatenate(train_x, axis=0)  
    train_y = np.concatenate(train_y, axis=0) 
    print("loading test...")
    dict_ = unpickle('./cifar-10-batches-py/test_batch')
    print('load finished')
    return train_x, train_y, np.array(dict_[b'data']), np.array(one_hot(dict_[b'labels'],10))
Xtrain, Ytrain, Xtest, Ytest = cifar10()  
print(Xtrain.shape , Ytrain.shape, Xtest.shape , Ytest.shape)

In [0]:
tf.reset_default_graph()

learning_rate=0.001
batch=500
entity=3072   #32*32*3
classes=10
num_epoch=100
data_length = Xtrain.shape[0]

x=tf.placeholder(tf.float32,[None,entity ])
y=tf.placeholder(tf.float32,[None,classes])

weights={
     'w1' :tf.Variable(tf.random_normal([5,5,3,32])),  #feature map 1
     'w2' :tf.Variable(tf.random_normal([5,5,32,64])) , #feature map 2
     'w3' :tf.Variable(tf.random_normal([8*8*64,1024])), # full concent layer
     'w4' :tf.Variable(tf.random_normal([1024 , classes]))  # output layer  
       }
biases={
      'b1' :tf.Variable(tf.random_normal([32])) , #feature map 1
      'b2'  :tf.Variable(tf.random_normal([64])), #feature map 2
      'b3' :tf.Variable(tf.random_normal([1024])), # full concent layer
      'b4' :tf.Variable(tf.random_normal([classes])) # output layer  
        }
def conv2d_step(x,w,b,stride=1):
    x=tf.nn.conv2d(x, w, strides=[1, stride, stride, 1], padding='SAME')
    return tf.add(x,b)
def detector_step(x):
    return tf.nn.relu(x)
def pool_step(x,stride=2,dtype='SAME'):
    return tf.nn.max_pool(x, ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
def convolution_net(x,w,b):
    x=tf.reshape(x,shape=[-1, 32, 32, 3])
    fearture_map1 = conv2d_step(x, weights['w1'], biases['b1'])
    feature1=detector_step(fearture_map1)
    hidden1=pool_step(feature1,stride=2,dtype='SAME')
    fearture_map2 = conv2d_step(hidden1,weights['w2'],biases['b2'])
    feature2=detector_step(fearture_map2)
    hidden2=pool_step(feature2,stride=2,dtype='SAME')
    
    full_connect_layer1=tf.reshape(hidden2, [-1, weights['w3'].get_shape().as_list()[0]])
    full_connect_layer1=tf.nn.relu(tf.add(tf.matmul(full_connect_layer1, weights['w3']),biases['b3']))
    out = tf.add(tf.matmul(full_connect_layer1,weights['w4']),biases['b4'])
    return out ,feature1 ,feature2 

predict, feature1, feature2=convolution_net(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=predict, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
prediction = tf.equal(tf.argmax(predict,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
print('construct model')


In [0]:
conv1,pool1,conv2, pool2=sess.run([feature1,hidden1,feature2,hidden1 ] ,\
                                  feed_dict={x:Xtrain[random_] ,y:Ytrain[random_]})

In [0]:

init=tf.global_variables_initializer()
session_conf=tf.ConfigProto(intra_op_parallelism_threads=4)
sess=tf.Session(config=session_conf)
sess.run(init)
print('start training')
for i in range(num_epoch) :
    for  j in range(int (data_length/batch)):
        batch_x, batch_y = Xtrain[batch*j:batch*(j+1)],Ytrain[batch*j:batch*(j+1)]
        sess.run(optimizer,feed_dict={x:batch_x,  y:batch_y})
    print (i,'train loss and accuracy : ',sess.run([cost,accuracy],feed_dict={x:Xtrain,y:Ytrain}))
    print(i,'test loss and accuracy : ',sess.run([cost,accuracy],feed_dict={x: Xtest ,y: Ytest}))

random_ = [5,6,40,15,32]
conv1,conv2=sess.run([feature1,feature2 ] ,\
                                  feed_dict={x:Xtrain[random_] ,y:Ytrain[random_]})
#sess.close()
image1 = conv1[1][0]
image2 = conv1[2][0]
image3 = conv2[2][0]
image4 = conv2[0][0]
#  
plt.figure(1)  
plt.imshow(image1)  
plt.figure(2)  
plt.imshow(image2)  

plt.figure(3)  
plt.imshow(image3)  
plt.figure(4)  
plt.imshow(image4)  

In [0]:
conv1,conv2=sess.run([feature1,feature2 ] ,\
                                  feed_dict={x:Xtrain[random_] ,y:Ytrain[random_]})
#sess.close()
image1 = conv1[0][0]
image2 = conv1[0][0]
image3 = conv2[0][0]
image4 = conv2[0][0]
#  
plt.figure(1)  
plt.imshow(image1)  
plt.figure(2)  
plt.imshow(image2)  

plt.figure(3)  
plt.imshow(image3)  
plt.figure(4)  
plt.imshow(image4)